In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/sgpjesus/bank-account-fraud-dataset-neurips-2022/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: arpithshanbhag
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sgpjesus/bank-account-fraud-dataset-neurips-2022


In [41]:
data = pd.read_csv("/content/bank-account-fraud-dataset-neurips-2022/Base.csv")



AttributeError: 'DataFrame' object has no attribute 'cor'

In [60]:
# import libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report


X = data.drop('fraud_bool', axis = 1)
y = data['fraud_bool']

# Let's only work with a few quantitative features

features = ['income',
       'customer_age',
       'credit_risk_score', 'proposed_credit_limit', 'name_email_similarity']


# Scaling data

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[features])
X_scaled_df = pd.DataFrame(X_scaled, columns=features, index=X.index)

# Add some categorical features

#cat = ['device_os','housing_status'] #, 'keep_alive_session', 'housing_status'

# One hot encode them

#X_cat_encode = pd.get_dummies(X[cat], drop_first=True, columns= cat)

#X_new = pd.concat([X_scaled_df,X_cat_encode], axis = 1)

#X_new.head()




,income,customer_age,credit_risk_score,proposed_credit_limit,name_email_similarity,device_os_macintosh,device_os_other,device_os_windows,device_os_x11,housing_status_BB,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG
0,-0.904778,0.524782,0.459380,2.01852,1.704497,False,False,False,False,False,True,False,False,False,False
1,0.817326,-1.138310,0.330221,2.01852,0.427954,False,True,False,False,False,True,False,False,False,False
2,0.817326,0.524782,-0.602591,-0.64782,1.739779,False,False,True,False,False,True,False,False,False,False
3,0.128484,-0.306764,-0.588240,-0.64782,-0.064312,False,False,False,False,False,True,False,False,False,False
4,1.161747,0.524782,-0.573889,-0.64782,1.205752,False,True,False,False,False,True,False,False,False,False


In [67]:
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, stratify=y , random_state=42)

In [68]:
model = GaussianNB()
model.fit(x_train, y_train)

GaussianNB()

In [69]:
y_pred = model.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

[[197335    459]
 [  2110     96]]
              precision    recall  f1-score   support

           0     0.9894    0.9977    0.9935    197794
           1     0.1730    0.0435    0.0695      2206

    accuracy                         0.9872    200000
   macro avg     0.5812    0.5206    0.5315    200000
weighted avg     0.9804    0.9872    0.9833    200000



In [72]:
# Let's try XGBoost

import xgboost as xgb
from xgboost import XGBClassifier

model1 =  XGBClassifier()
model1.fit(x_train,y_train)

y_pred1 = model1.predict(x_test)

print(confusion_matrix(y_test,y_pred1))
print(classification_report(y_test,y_pred1))

[[197786      8]
 [  2202      4]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    197794
           1       0.33      0.00      0.00      2206

    accuracy                           0.99    200000
   macro avg       0.66      0.50      0.50    200000
weighted avg       0.98      0.99      0.98    200000



In [99]:
# After these atrocious results, let's try PCA.

from sklearn.decomposition import PCA

X_num = data.select_dtypes(include = [np.number])
X_num = X_num.drop(columns = ['fraud_bool'])
X_num_scaled = scaler.fit_transform(X_num)


pca = PCA()
X_pca = pca.fit_transform(X_num_scaled)

X_train, X_test,Y_train,Y_test = train_test_split(X_pca,y,stratify=y, random_state=45)

In [100]:
model2 = GaussianNB()
model2.fit(X_train, Y_train)

GaussianNB()

In [101]:
Y_pred = model2.predict(X_test)
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))


[[237651   9592]
 [  2360    397]]
              precision    recall  f1-score   support

           0       0.99      0.96      0.98    247243
           1       0.04      0.14      0.06      2757

    accuracy                           0.95    250000
   macro avg       0.51      0.55      0.52    250000
weighted avg       0.98      0.95      0.97    250000

